<a href="https://colab.research.google.com/github/Juthathip-Saetan/Kaggle_Dataset_Titanic_V2/blob/main/Kaggle_Dataset_Titanic_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Kaggle-Titanic-Dataset---Machine-Learning-from-Disaster**
Predict survival on the Titanic and get familiar with ML basics

**The outline of our project**
+ Drop some unnecessary columns

+ Fill in missing data

+ Encode the data

+ Train XGBoost model with GridSearchCV on training set

+ Train Random Forest model.

Download the dataset to the program.


In [ ]:
import pandas as pd
Dataset = pd.read_csv('/content/train.csv')

Dataset

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


Divide the dataset into X and y.

In [ ]:
X = Dataset.drop(['Survived'], axis=1)
X.shape

(891, 11)

In [ ]:
y = Dataset['Survived']
y.shape

(891,)

Split X and y to training and test set.



In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =0.2)

In [ ]:
X_train.shape

(712, 11)

In [ ]:
X_test.shape

(179, 11)

Look at some information of the training set and check null values.

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 72 to 520
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Pclass       712 non-null    int64  
 2   Name         712 non-null    object 
 3   Sex          712 non-null    object 
 4   Age          572 non-null    float64
 5   SibSp        712 non-null    int64  
 6   Parch        712 non-null    int64  
 7   Ticket       712 non-null    object 
 8   Fare         712 non-null    float64
 9   Cabin        163 non-null    object 
 10  Embarked     710 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 66.8+ KB


Check the number of unique items in columns.

In [ ]:
X_train.isnull().sum()/len(X_train)

PassengerId    0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.196629
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Cabin          0.771067
Embarked       0.002809
dtype: float64

Drop some columns that we think they are unnecessary.

In [ ]:
X_train.drop(['PassengerId','Name','Ticket'],axis=1, inplace=True)

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 72 to 520
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Sex       712 non-null    object 
 2   Age       572 non-null    float64
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Cabin     163 non-null    object 
 7   Embarked  710 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 50.1+ KB


In [ ]:
X_train['Age'].unique()

array([21.  , 38.  , 28.  , 17.  , 56.  , 18.  , 34.  , 20.  , 71.  ,
       47.  , 44.  , 65.  , 27.  , 25.  , 28.5 ,   nan, 22.  , 39.  ,
        2.  , 50.  , 16.  , 30.  , 42.  ,  7.  , 20.5 , 37.  , 36.  ,
       26.  , 35.  , 40.  , 62.  , 19.  , 58.  ,  9.  , 74.  , 45.  ,
       32.  ,  1.  ,  5.  , 33.  , 49.  ,  3.  , 31.  , 29.  , 43.  ,
       41.  , 61.  , 23.  , 40.5 , 52.  , 15.  , 46.  , 48.  , 32.5 ,
       54.  , 24.  , 51.  , 55.  ,  8.  , 34.5 , 66.  ,  6.  ,  4.  ,
       11.  , 13.  , 60.  ,  0.67,  0.75, 14.  , 63.  ,  0.83, 45.5 ,
       59.  , 57.  , 30.5 , 55.5 , 24.5 , 70.  , 36.5 , 70.5 , 14.5 ,
       12.  , 64.  ,  0.92])

In [ ]:
X_train['Cabin'].unique()

array([nan, 'C50', 'C93', 'D46', 'B4', 'C95', 'E121', 'F2', 'D19', 'D35',
       'C46', 'A10', 'C45', 'E34', 'C118', 'C91', 'B49', 'B80', 'B18',
       'E25', 'B28', 'B77', 'T', 'E36', 'B38', 'C92', 'C148', 'C78',
       'E10', 'A32', 'B20', 'C23 C25 C27', 'B3', 'E24', 'C68', 'B19',
       'B42', 'C70', 'A5', 'B78', 'A16', 'B30', 'D6', 'C123', 'E46',
       'D28', 'D47', 'C82', 'B86', 'C99', 'B39', 'C65', 'C22 C26', 'E38',
       'D36', 'B5', 'B94', 'A20', 'C2', 'D26', 'G6', 'C54', 'E40', 'C83',
       'D30', 'C85', 'D', 'A26', 'B37', 'B50', 'F33', 'C86', 'E44',
       'D10 D12', 'B35', 'E33', 'D37', 'C7', 'F38', 'B57 B59 B63 B66',
       'C47', 'E17', 'B71', 'F4', 'D7', 'E8', 'E101', 'E58',
       'B51 B53 B55', 'D20', 'B102', 'C111', 'D33', 'B58 B60', 'C125',
       'E31', 'A14', 'B96 B98', 'C62 C64', 'D9', 'C126', 'D15', 'B22',
       'E63', 'B82 B84', 'C52', 'E50', 'C90', 'B69', 'C124', 'C128',
       'A24', 'E49', 'D17', 'D45', 'C110', 'D21', 'C49', 'A6', 'C106',
       'E68', 'C3

In [ ]:
for column in X_train.columns:
    print(column, len(X_train[column].unique()))

Pclass 3
Sex 2
Age 84
SibSp 7
Parch 6
Fare 225
Cabin 127
Embarked 4


Fill in missing value.

In [ ]:
float64_column = ['Age','Pclass','SibSp','Parch','Fare']
object_cabin_column = ['Cabin']
object_embarked_column = ['Embarked','Sex']

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 72 to 520
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Sex       712 non-null    object 
 2   Age       572 non-null    float64
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Cabin     163 non-null    object 
 7   Embarked  710 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 50.1+ KB


In [ ]:
object_embarked_column

['Embarked', 'Sex']

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np

imp_float64_constant = SimpleImputer(missing_values= np.nan, strategy='constant',fill_value=-1)
imp_object_constant = SimpleImputer(missing_values= np.nan, strategy='constant',fill_value=None)
imp_mode = SimpleImputer(missing_values= np.nan, strategy='most_frequent')

In [ ]:
imp_mode.fit(X_train[object_embarked_column])

SimpleImputer(strategy='most_frequent')

In [ ]:
imp_mode.feature_names_in_

array(['Embarked', 'Sex'], dtype=object)

In [ ]:
imp_mode.statistics_

array(['S', 'male'], dtype=object)

In [ ]:
X_train[object_embarked_column] = imp_mode.transform(X_train[object_embarked_column])

In [ ]:
X_train[object_embarked_column].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 72 to 520
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Embarked  712 non-null    object
 1   Sex       712 non-null    object
dtypes: object(2)
memory usage: 16.7+ KB


In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 72 to 520
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Sex       712 non-null    object 
 2   Age       572 non-null    float64
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Cabin     163 non-null    object 
 7   Embarked  712 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 50.1+ KB


In [ ]:
imp_float64_constant.fit(X_train[float64_column])

SimpleImputer(fill_value=-1, strategy='constant')

In [ ]:
imp_float64_constant.statistics_

array([-1., -1., -1., -1., -1.])

In [ ]:
X_train[float64_column] = imp_float64_constant.transform(X_train[float64_column])

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 72 to 520
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    float64
 1   Sex       712 non-null    object 
 2   Age       712 non-null    float64
 3   SibSp     712 non-null    float64
 4   Parch     712 non-null    float64
 5   Fare      712 non-null    float64
 6   Cabin     163 non-null    object 
 7   Embarked  712 non-null    object 
dtypes: float64(5), object(3)
memory usage: 50.1+ KB


In [ ]:
imp_object_constant.fit(X_train[object_cabin_column])

SimpleImputer(strategy='constant')

In [ ]:
imp_object_constant.statistics_

array(['missing_value'], dtype=object)

In [ ]:
X_train[object_cabin_column] = imp_object_constant.transform(X_train[object_cabin_column])

In [ ]:
X_train[object_cabin_column]

,Cabin
72,missing_value
822,missing_value
403,missing_value
433,missing_value
879,C50
...,...
449,C104
305,C22 C26
625,D50
85,missing_value


In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 72 to 520
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    float64
 1   Sex       712 non-null    object 
 2   Age       712 non-null    float64
 3   SibSp     712 non-null    float64
 4   Parch     712 non-null    float64
 5   Fare      712 non-null    float64
 6   Cabin     712 non-null    object 
 7   Embarked  712 non-null    object 
dtypes: float64(5), object(3)
memory usage: 50.1+ KB


In [ ]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
72,2.0,male,21.00,0.0,0.0,73.5000,missing_value,S
822,1.0,male,38.00,0.0,0.0,0.0000,missing_value,S
403,3.0,male,28.00,1.0,0.0,15.8500,missing_value,S
433,3.0,male,17.00,0.0,0.0,7.1250,missing_value,S
879,1.0,female,56.00,0.0,1.0,83.1583,C50,C
...,...,...,...,...,...,...,...,...
449,1.0,male,52.00,0.0,0.0,30.5000,C104,S
305,1.0,male,0.92,1.0,2.0,151.5500,C22 C26,S
625,1.0,male,61.00,0.0,0.0,32.3208,D50,S
85,3.0,female,33.00,3.0,0.0,15.8500,missing_value,S


In [ ]:
def cabin_sep(object_cabin_column):
    cabin_type = []

    mhee = object_cabin_column.copy()
    mhee = mhee.reset_index( drop=True )

    for i in range(len(mhee)):

            if mhee[i] == 'missing_value': 
                cabin_type.append('NaN') 
            else:    
                cabin_type.append(mhee[i][:1]) 
            
    return cabin_type

In [ ]:
X_train['cabin_type'] = cabin_sep(X_train.Cabin)

In [ ]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,cabin_type
72,2.0,male,21.00,0.0,0.0,73.5000,missing_value,S,NaN
822,1.0,male,38.00,0.0,0.0,0.0000,missing_value,S,NaN
403,3.0,male,28.00,1.0,0.0,15.8500,missing_value,S,NaN
433,3.0,male,17.00,0.0,0.0,7.1250,missing_value,S,NaN
879,1.0,female,56.00,0.0,1.0,83.1583,C50,C,C
...,...,...,...,...,...,...,...,...,...
449,1.0,male,52.00,0.0,0.0,30.5000,C104,S,C
305,1.0,male,0.92,1.0,2.0,151.5500,C22 C26,S,C
625,1.0,male,61.00,0.0,0.0,32.3208,D50,S,D
85,3.0,female,33.00,3.0,0.0,15.8500,missing_value,S,NaN


In [ ]:
X_train.drop(['Cabin'],axis=1, inplace=True)

In [ ]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,cabin_type
72,2.0,male,21.00,0.0,0.0,73.5000,S,NaN
822,1.0,male,38.00,0.0,0.0,0.0000,S,NaN
403,3.0,male,28.00,1.0,0.0,15.8500,S,NaN
433,3.0,male,17.00,0.0,0.0,7.1250,S,NaN
879,1.0,female,56.00,0.0,1.0,83.1583,C,C
...,...,...,...,...,...,...,...,...
449,1.0,male,52.00,0.0,0.0,30.5000,S,C
305,1.0,male,0.92,1.0,2.0,151.5500,S,C
625,1.0,male,61.00,0.0,0.0,32.3208,S,D
85,3.0,female,33.00,3.0,0.0,15.8500,S,NaN


Lets encode the data

In [ ]:
ordinal_columns = ['Sex']
nominal_columns = ['Embarked','cabin_type']

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_enc = OrdinalEncoder()

In [ ]:
ordinal_enc.fit(X_train[ordinal_columns])

OrdinalEncoder()

In [ ]:
ordinal_enc.feature_names_in_

array(['Sex'], dtype=object)

In [ ]:
 ordinal_enc.categories_

[array(['female', 'male'], dtype=object)]

In [ ]:
X_train[ordinal_columns] = ordinal_enc.transform(X_train[ordinal_columns])
X_train[ordinal_columns]

,Sex
72,1.0
822,1.0
403,1.0
433,1.0
879,0.0
...,...
449,1.0
305,1.0
625,1.0
85,0.0


In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 72 to 520
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      712 non-null    float64
 1   Sex         712 non-null    float64
 2   Age         712 non-null    float64
 3   SibSp       712 non-null    float64
 4   Parch       712 non-null    float64
 5   Fare        712 non-null    float64
 6   Embarked    712 non-null    object 
 7   cabin_type  712 non-null    object 
dtypes: float64(6), object(2)
memory usage: 50.1+ KB


In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehot_enc = OneHotEncoder(handle_unknown='ignore')

onehot_enc.fit(X_train[nominal_columns])

OneHotEncoder(handle_unknown='ignore')

In [ ]:
onehot_enc.feature_names_in_

array(['Embarked', 'cabin_type'], dtype=object)

In [ ]:
onehot_enc.categories_

[array(['C', 'Q', 'S'], dtype=object),
 array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'NaN', 'T'], dtype=object)]

In [ ]:
onehot_enc.get_feature_names_out()

array(['Embarked_C', 'Embarked_Q', 'Embarked_S', 'cabin_type_A',
       'cabin_type_B', 'cabin_type_C', 'cabin_type_D', 'cabin_type_E',
       'cabin_type_F', 'cabin_type_G', 'cabin_type_NaN', 'cabin_type_T'],
      dtype=object)

In [ ]:
encoded_nominal_columns = onehot_enc.get_feature_names_out()
encoded_nominal_columns

array(['Embarked_C', 'Embarked_Q', 'Embarked_S', 'cabin_type_A',
       'cabin_type_B', 'cabin_type_C', 'cabin_type_D', 'cabin_type_E',
       'cabin_type_F', 'cabin_type_G', 'cabin_type_NaN', 'cabin_type_T'],
      dtype=object)

In [ ]:
X_train[encoded_nominal_columns] = onehot_enc.transform(X_train[nominal_columns]).toarray()
X_train[encoded_nominal_columns]

,Embarked_C,Embarked_Q,Embarked_S,cabin_type_A,cabin_type_B,cabin_type_C,cabin_type_D,cabin_type_E,cabin_type_F,cabin_type_G,cabin_type_NaN,cabin_type_T
72,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
822,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
403,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
433,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
879,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
449,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
305,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
625,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
85,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 72 to 520
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Pclass          712 non-null    float64
 1   Sex             712 non-null    float64
 2   Age             712 non-null    float64
 3   SibSp           712 non-null    float64
 4   Parch           712 non-null    float64
 5   Fare            712 non-null    float64
 6   Embarked        712 non-null    object 
 7   cabin_type      712 non-null    object 
 8   Embarked_C      712 non-null    float64
 9   Embarked_Q      712 non-null    float64
 10  Embarked_S      712 non-null    float64
 11  cabin_type_A    712 non-null    float64
 12  cabin_type_B    712 non-null    float64
 13  cabin_type_C    712 non-null    float64
 14  cabin_type_D    712 non-null    float64
 15  cabin_type_E    712 non-null    float64
 16  cabin_type_F    712 non-null    float64
 17  cabin_type_G    712 non-null    fl

In [ ]:
X_train.drop(nominal_columns,axis=1,inplace=True)

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 72 to 520
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Pclass          712 non-null    float64
 1   Sex             712 non-null    float64
 2   Age             712 non-null    float64
 3   SibSp           712 non-null    float64
 4   Parch           712 non-null    float64
 5   Fare            712 non-null    float64
 6   Embarked_C      712 non-null    float64
 7   Embarked_Q      712 non-null    float64
 8   Embarked_S      712 non-null    float64
 9   cabin_type_A    712 non-null    float64
 10  cabin_type_B    712 non-null    float64
 11  cabin_type_C    712 non-null    float64
 12  cabin_type_D    712 non-null    float64
 13  cabin_type_E    712 non-null    float64
 14  cabin_type_F    712 non-null    float64
 15  cabin_type_G    712 non-null    float64
 16  cabin_type_NaN  712 non-null    float64
 17  cabin_type_T    712 non-null    fl

In [ ]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,cabin_type_A,cabin_type_B,cabin_type_C,cabin_type_D,cabin_type_E,cabin_type_F,cabin_type_G,cabin_type_NaN,cabin_type_T
72,2.0,1.0,21.00,0.0,0.0,73.5000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
822,1.0,1.0,38.00,0.0,0.0,0.0000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
403,3.0,1.0,28.00,1.0,0.0,15.8500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
433,3.0,1.0,17.00,0.0,0.0,7.1250,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
879,1.0,0.0,56.00,0.0,1.0,83.1583,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,1.0,1.0,52.00,0.0,0.0,30.5000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
305,1.0,1.0,0.92,1.0,2.0,151.5500,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
625,1.0,1.0,61.00,0.0,0.0,32.3208,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
85,3.0,0.0,33.00,3.0,0.0,15.8500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Create and train the model XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

xgb = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)

params_xgb = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(100, 210, 10),
    'learning_rate': [0.1, 0.01, 0.05]
}

gs_xgb = GridSearchCV(estimator=xgb , param_grid=params_xgb, cv=5, verbose=3, scoring='accuracy')

In [ ]:
gs_xgb.fit(X_train,y_train)

Fitting 5 folds for each of 264 candidates, totalling 1320 fits
[CV 1/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.706 total time=   0.1s
[CV 2/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.762 total time=   0.1s
[CV 3/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.845 total time=   0.1s
[CV 4/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.817 total time=   0.1s
[CV 5/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.810 total time=   0.1s
[CV 1/5] END learning_rate=0.1, max_depth=2, n_estimators=110;, score=0.720 total time=   0.1s
[CV 2/5] END learning_rate=0.1, max_depth=2, n_estimators=110;, score=0.762 total time=   0.1s
[CV 3/5] END learning_rate=0.1, max_depth=2, n_estimators=110;, score=0.852 total time=   0.1s
[CV 4/5] END learning_rate=0.1, max_depth=2, n_estimators=110;, score=0.831 total time=   0.1s
[CV 5/5] END learning_rate=0.1, max_depth=2, n_estimators=110;, score=0.817 total

GridSearchCV(cv=5, estimator=XGBClassifier(nthread=4, seed=42),
             param_grid={'learning_rate': [0.1, 0.01, 0.05],
                         'max_depth': range(2, 10),
                         'n_estimators': range(100, 210, 10)},
             scoring='accuracy', verbose=3)

In [ ]:
gs_xgb.best_params_

{'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 120}

In [ ]:
gs_xgb.best_score_

0.8189402147148627

In [ ]:
gs_xgb.best_estimator_

XGBClassifier(learning_rate=0.05, max_depth=7, n_estimators=120, nthread=4,
              seed=42)

Let's see the accuracy score on train set.

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_train,gs_xgb.predict(X_train))

0.925561797752809

Do the feature engineering process to X_test.

In [ ]:
X_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
579,580,3,"Jussila, Mr. Eiriik",male,32.0,0,0,STON/O 2. 3101286,7.9250,NaN,S
235,236,3,"Harknett, Miss. Alice Phoebe",female,NaN,0,0,W./C. 6609,7.5500,NaN,S
682,683,3,"Olsvigen, Mr. Thor Anderson",male,20.0,0,0,6563,9.2250,NaN,S
398,399,2,"Pain, Dr. Alfred",male,23.0,0,0,244278,10.5000,NaN,S
531,532,3,"Toufik, Mr. Nakli",male,NaN,0,0,2641,7.2292,NaN,C
...,...,...,...,...,...,...,...,...,...,...,...
687,688,3,"Dakic, Mr. Branko",male,19.0,0,0,349228,10.1708,NaN,S
837,838,3,"Sirota, Mr. Maurice",male,NaN,0,0,392092,8.0500,NaN,S
421,422,3,"Charters, Mr. David",male,21.0,0,0,A/5. 13032,7.7333,NaN,Q
681,682,1,"Hassab, Mr. Hammad",male,27.0,0,0,PC 17572,76.7292,D49,C


In [ ]:
X_test.drop(['PassengerId','Ticket','Name'],axis=1,inplace=True)

In [ ]:
X_test[float64_column] = imp_float64_constant.transform(X_test[float64_column])
X_test[object_embarked_column] = imp_mode.transform(X_test[object_embarked_column])
X_test[object_cabin_column] = imp_object_constant.transform(X_test[object_cabin_column])

In [ ]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 579 to 547
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    179 non-null    float64
 1   Sex       179 non-null    object 
 2   Age       179 non-null    float64
 3   SibSp     179 non-null    float64
 4   Parch     179 non-null    float64
 5   Fare      179 non-null    float64
 6   Cabin     179 non-null    object 
 7   Embarked  179 non-null    object 
dtypes: float64(5), object(3)
memory usage: 12.6+ KB


In [ ]:
X_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
579,3.0,male,32.0,0.0,0.0,7.9250,missing_value,S
235,3.0,female,-1.0,0.0,0.0,7.5500,missing_value,S
682,3.0,male,20.0,0.0,0.0,9.2250,missing_value,S
398,2.0,male,23.0,0.0,0.0,10.5000,missing_value,S
531,3.0,male,-1.0,0.0,0.0,7.2292,missing_value,C
...,...,...,...,...,...,...,...,...
687,3.0,male,19.0,0.0,0.0,10.1708,missing_value,S
837,3.0,male,-1.0,0.0,0.0,8.0500,missing_value,S
421,3.0,male,21.0,0.0,0.0,7.7333,missing_value,Q
681,1.0,male,27.0,0.0,0.0,76.7292,D49,C


In [ ]:
X_test['cabin_type'] = cabin_sep(X_test.Cabin)

In [ ]:
X_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,cabin_type
579,3.0,male,32.0,0.0,0.0,7.9250,missing_value,S,NaN
235,3.0,female,-1.0,0.0,0.0,7.5500,missing_value,S,NaN
682,3.0,male,20.0,0.0,0.0,9.2250,missing_value,S,NaN
398,2.0,male,23.0,0.0,0.0,10.5000,missing_value,S,NaN
531,3.0,male,-1.0,0.0,0.0,7.2292,missing_value,C,NaN
...,...,...,...,...,...,...,...,...,...
687,3.0,male,19.0,0.0,0.0,10.1708,missing_value,S,NaN
837,3.0,male,-1.0,0.0,0.0,8.0500,missing_value,S,NaN
421,3.0,male,21.0,0.0,0.0,7.7333,missing_value,Q,NaN
681,1.0,male,27.0,0.0,0.0,76.7292,D49,C,D


In [ ]:
X_test.drop(['Cabin'],axis=1, inplace=True)

In [ ]:
X_test[ordinal_columns] = ordinal_enc.transform(X_test[ordinal_columns])
X_test[encoded_nominal_columns] = onehot_enc.transform(X_test[nominal_columns]).toarray()

X_test = X_test.drop(nominal_columns,axis=1)
X_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,cabin_type_A,cabin_type_B,cabin_type_C,cabin_type_D,cabin_type_E,cabin_type_F,cabin_type_G,cabin_type_NaN,cabin_type_T
579,3.0,1.0,32.0,0.0,0.0,7.9250,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
235,3.0,0.0,-1.0,0.0,0.0,7.5500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
682,3.0,1.0,20.0,0.0,0.0,9.2250,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
398,2.0,1.0,23.0,0.0,0.0,10.5000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
531,3.0,1.0,-1.0,0.0,0.0,7.2292,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,3.0,1.0,19.0,0.0,0.0,10.1708,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
837,3.0,1.0,-1.0,0.0,0.0,8.0500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
421,3.0,1.0,21.0,0.0,0.0,7.7333,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
681,1.0,1.0,27.0,0.0,0.0,76.7292,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


Let's see the accuracy score on test set.

In [ ]:
gs_xgb.predict(X_test)

array([0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0])

In [ ]:
from sklearn.metrics import accuracy_score 
accuracy_score(y_test,gs_xgb.predict(X_test))

0.888268156424581

In [ ]:
y_test

579    1
235    0
682    0
398    0
531    0
      ..
687    0
837    0
421    0
681    1
547    1
Name: Survived, Length: 179, dtype: int64

Let''s load the kaggle test data (without target y).

In [ ]:
import pandas as pd
X_realworld = pd.read_csv('/content/test.csv')

X_realworld

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [ ]:
X_realworld.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)

In [ ]:
X_realworld[float64_column] = imp_float64_constant.transform(X_realworld[float64_column])
X_realworld[object_embarked_column] = imp_mode.transform(X_realworld[object_embarked_column])
X_realworld[object_cabin_column] = imp_object_constant.transform(X_realworld[object_cabin_column])

In [ ]:
X_realworld['cabin_type'] = cabin_sep(X_realworld.Cabin)

In [ ]:
X_realworld.drop(['Cabin'],axis=1, inplace=True)

In [ ]:
X_realworld[ordinal_columns] = ordinal_enc.transform(X_realworld[ordinal_columns])
X_realworld[encoded_nominal_columns] = onehot_enc.transform(X_realworld[nominal_columns]).toarray()

X_realworld = X_realworld.drop(nominal_columns,axis=1)
X_realworld

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,cabin_type_A,cabin_type_B,cabin_type_C,cabin_type_D,cabin_type_E,cabin_type_F,cabin_type_G,cabin_type_NaN,cabin_type_T
0,3.0,1.0,34.5,0.0,0.0,7.8292,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,3.0,0.0,47.0,1.0,0.0,7.0000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2.0,1.0,62.0,0.0,0.0,9.6875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3.0,1.0,27.0,0.0,0.0,8.6625,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,3.0,0.0,22.0,1.0,1.0,12.2875,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,3.0,1.0,-1.0,0.0,0.0,8.0500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
414,1.0,0.0,39.0,0.0,0.0,108.9000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
415,3.0,1.0,38.5,0.0,0.0,7.2500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
416,3.0,1.0,-1.0,0.0,0.0,8.0500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
gs_xgb.predict(X_realworld)

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

Load the submisssion file to create our own prediction file (using our model).

In [ ]:
gender_submission = pd.read_csv('/content/gender_submission.csv')

In [ ]:
gs_xgb_submission = gender_submission.copy()

In [ ]:
gender_submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


Use our model to predict the Kaggle test data.

In [ ]:
gs_xgb_submission['Survived'] = gs_xgb.predict(X_realworld)
gs_xgb_submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
gs_xgb_submission

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

Save the excel file, then finished.

In [ ]:
gs_xgb_submission.to_csv('/content/gs_xgb_submission_ver2.csv',index=False)

Let try Random Forest model.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100, max_depth=6,random_state=5)

In [ ]:
random_forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=6, random_state=5)

In [ ]:
Y_pred1 = random_forest.predict(X_test)

In [ ]:
random_forest.score(X_train, y_train)

0.8637640449438202

In [ ]:
acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)

In [ ]:
acc_random_forest

86.38

In [ ]:
accuracy_score(y_test,Y_pred1)

0.8435754189944135

In [ ]:
random_forest.predict(X_realworld)

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [ ]:
random_forest_submission = gender_submission.copy()

In [ ]:
random_forest_submission['Survived'] = random_forest.predict(X_realworld)
random_forest_submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
random_forest_submission.to_csv('/content/Random_forest_submissionVer2.csv',index=False)